In [4]:
import warnings
import re
warnings.filterwarnings("ignore", message=".*The 'nopython' keyword.*")
warnings.filterwarnings('always', category=DeprecationWarning, module=r'^{0}\.'.format(re.escape(__name__)))
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)


In [5]:
roles = ['School_Administrators', 'Clinical_Staff', 'School_Counselor', 'School Psychologist', 'Teachers']
context = '''Using the information contained in the context,give a comprehensive answer to the question.
                         Respond only to the question asked, response should be concise and relevant to the question. 
                         Provide the number of the source document when relevant. 
                         If the answer cannot be deduced from the context, do not give an answer. 

                        We will provide you the context from the course content of Multi-Tiered System of Supports (MTSS) 
                        a comprehensive, evidence-based framework for
                        supporting successful student outcomes and eliminating barriers to learning. It is designed to
                        identify and address the needs of all students, including those who may be struggling
                        academically and behaviorally, by providing interventions and supports at varying levels of
                        intensity.'''
system_templates = {
            "School_Administrators": """You have been asked a question by School Administrators who job role is to ensures there is an MTSS team to design the school-wide implementation process, progress monitoring protocols, and data collection procedures. 
							Below are the context:\nThis is context1:\n{context1}\n\nThis is context2:\n{context2}\n\nThis is context3:\n{context3}\n\nThis is context4:\n{context4}""",
"Clinical_Staff": """You have been asked a question by Clinical_Staff who job role is to ensures, Clinical_Staff is a key member of both the MTSS team and the
					school culture. Their visibility and voice on this team and throughout the building
					communicates the importance of mental health across all tiers of the MTSS.  They
					function as not only a referral source for students and families with more intensive
					needs, but also as a consultant for wellness efforts across the tiers. 
					School mental health clinicians:
					- attend all MTSS meetings
					-cprepare and report on qualitative and quantitative progress data on students and families receiving intensive support
					- facilitate problem solving, offering key insights on the impacts and potential root causes of internalizing and externalizing behaviors
					- identify and collaborate on research-based intervention strategies implemented by school staff
					- support problem-solving and mediation for educators 
					- lead and plan professional development related to individual and systems level mental health and wellness strategies 
					- facilitate small groups related to prevention, intervention, and postvention support
					- continuously interact with and engage families at both the community and individual level
					Below are the context:\nThis is context1:\n{context1}\n\nThis is context2:\n{context2}\n\nThis is context3:\n{context3}\n\nThis is context4:\n{context4}""",
"School_Counselor": """You have been asked a question by School Counselors who job role is to As a primary resource for administrators, teachers, and parents regarding mental health awareness, school counselors implement school counseling programs addressing the needs of all students. 
						School counselors deliver instruction, appraisal, and advisement to students in all tiers and collaborate with other specialized instructional and intervention personnel, educators, and families to ensure appropriate academic and behavioral supports for students within the school’s MTSS framework. 
						According to the American School Counselor Association, school counselors align
							with the school’s MTSS by:
							- providing all students with standards-based school counseling instruction to address universal academic, career, and social/emotional development and
							analyzing academic, career, and social/emotional development data to identify students who need support
							- identifying and collaborating on research-based intervention strategies implemented by school staff
							- evaluating academic and behavioral progress after interventions
							- revising interventions as appropriate
							- referring to school and community services as appropriate
							- collaborating with administrators, teachers, other school professionals, community agencies, and families in MTSS design and implementation 
							- advocating for equitable education for all students and working to remove systemic barriers
						Below are the context:\nThis is context1:\n{context1}\n\nThis is context2:\n{context2}\n\nThis is context3:\n{context3}\n\nThis is context4:\n{context4}""",
'School Psychologist':"""You have been asked a question by School Administrators who job role is to School psychologists play an integral role in promoting and supporting competency
						development within the core components of MTSS, including data-informed decision
						making, evidence-based interventions, implementation ﬁdelity, and consultation
						and collaboration (National Association of School Psychologists, 2020). In MTSS,
						data-informed decision making includes universal screening of all students,
						implementation of evidenced-based interventions at multiple tiers, and ongoing
						progress monitoring to inform the decisions at each tier. A problem-solving process
						supports ongoing evaluation of the data in order to make timely and ongoing
						informed decisions (Gresham, 2007).  
						School psychologists 
						- contribute expertise in data interpretation and analysis, progress monitoring, and effective problem-solving.  
						- administer diagnostic screening assessments assist in observing students in the
						- instructional environment assist in designing interventions matched to student needs, based on data  
						- assist with the identiﬁcation of appropriate interventions and progress monitoring  
						- consult with the school-based leadership team and school staff regarding MTSS needs 
						- provide consultation and support to the school throughout the problem-solving phases.
						Below are the context:\nThis is context1:\n{context1}\n\nThis is context2:\n{context2}\n\nThis is context3:\n{context3}\n\nThis is context4:\n{context4}""",
'Teachers':"""You have been asked a question by School Administrators who job role is to
			- provide high-quality standard-based instruction and interventions with ﬁdelity  
			- implement selected schoolwide evidenced-based practices with ﬁdelity  
			- collect data on the effectiveness of Tier 1, Tier 2, and Tier 3 interventions
			(progress monitoring)  
			- collaborate in problem-solving efforts to determine interventions and supports  
			- implement strategies, support, and plans for small group and individual students
			- ensure that appropriate data are
			Below are the context:\nThis is context1:\n{context1}\n\nThis is context2:\n{context2}\n\nThis is context3:\n{context3}\n\nThis is context4:\n{context4}"""
            
        }

In [6]:
f = open('/Users/bytedance/Documents/Hackathon_2024/SBHA/openai_key.txt')

In [7]:
os.environ['OPENAI_API_KEY'] = f.read()
chat = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'], model='gpt-4-turbo',temperature=0)

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

loader = TextLoader('/Users/bytedance/Documents/Hackathon_2024/SBHA/Final_txt_document_course.txt')
data = loader.load()

MARKDOWN_SEPARATORS = [
"#####",
"*****",
"@@@@@",
"\n\n\n\n\n\n",
"\n\n\n\n\n",  
"\n\n\n\n",  
"\n\n\n",  
"\n\n",
    "\n",
    " ",
    "",
]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=20000,  # The maximum number of characters in a chunk: we selected this value arbitrarily
            chunk_overlap=100,  # The number of characters to overlap between chunks
            strip_whitespace=True,  # If `True`, strips whitespace from the start and end of every document
            separators=MARKDOWN_SEPARATORS,
        )

docs_processed = text_splitter.split_documents(data)
KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
            docs_processed, embeddings, distance_strategy=DistanceStrategy.COSINE
        )


## School_Administrators

In [16]:
user_query = "Briefly describe MTSS and the process of integrating behavioral health support into an existing academic framework."
role = 'School_Administrators'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


MTSS, or Multi-Tiered System of Supports, is a comprehensive, evidence-based framework designed to meet the diverse needs of all students through layered levels of support. It aims to enhance academic, behavioral, and social-emotional outcomes by providing interventions at varying intensities tailored to student needs. MTSS is structured around three tiers: Tier 1 (universal supports for all students), Tier 2 (targeted supports for students who need more assistance), and Tier 3 (intensive supports for individual students with significant needs).

Integrating behavioral health support into an existing academic MTSS involves a systematic approach that recognizes the importance of addressing both academic and mental health needs to support overall student well-being. This integration process typically includes:

1. **Assessing the Current State**: Reviewing the existing academic MTSS framework and supports for behavioral and mental health to identify gaps and strengths.

2. **Establishing

In [17]:
user_query = "Discuss the importance of integrating behavioral health into a tiered academic support framework."
role = 'School_Administrators'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

Integrating behavioral health into a tiered academic support framework, such as the Multi-Tiered System of Supports (MTSS), is crucial for several reasons. This integration not only enhances the overall effectiveness of educational interventions but also addresses the comprehensive needs of students, promoting both academic success and emotional well-being. Here are some key points highlighting the importance of this integration:

1. **Holistic Student Support**: Behavioral health is a critical component of overall student well-being. By integrating behavioral health into a tiered support framework, schools can provide a more holistic approach that addresses both academic and non-academic barriers to learning. This approach recognizes that students' emotional and mental health needs significantly impact their ability to learn and succeed academically.

2. **Early Identification and Intervention**: A tiered framework allows for the early identification of behavioral and emotional issues

In [18]:
user_query = "Discuss the barriers to implement an integrated MTSS framework."
role = 'School_Administrators'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

Implementing an integrated Multi-Tiered System of Supports (MTSS) framework involves a comprehensive approach that combines academic, behavioral, and social-emotional supports into a cohesive system. While this integration can significantly enhance student outcomes, several barriers can impede successful implementation. Here are some of the common challenges:

1. **Lack of Understanding and Buy-in:**
   - **Barrier:** Stakeholders, including staff and families, may not fully understand the purpose and benefits of an integrated MTSS framework. Without a clear understanding, there may be resistance to change and a lack of commitment.
   - **Solution:** Provide comprehensive training and ongoing professional development to all stakeholders. Regular communication about the goals, processes, and successes of MTSS can help build buy-in and understanding.

2. **Inadequate Resources:**
   - **Barrier:** Implementing MTSS requires sufficient resources, including funding, personnel, and material

In [19]:
user_query = "What are the key concepts in implementing an effective integrated MTSS?"
role = 'School_Administrators'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

Implementing an effective integrated Multi-Tiered System of Supports (MTSS) involves several key concepts that ensure the framework is comprehensive and addresses both academic and behavioral/mental health needs of students. Here are the essential concepts:

1. **Leadership and Team Structure**: Establish a dedicated leadership team that includes administrators, teachers, counselors, and other stakeholders. This team is responsible for overseeing the MTSS implementation, ensuring fidelity, and facilitating communication among all participants.

2. **Shared Vision and Goals**: Develop a clear and shared vision for MTSS implementation that includes specific, measurable goals. This shared vision helps align the efforts of all stakeholders and guides decision-making processes.

3. **Assessment of Readiness and Capacity**: Evaluate the school or district’s readiness and capacity to implement MTSS. This includes assessing available resources, staff competencies, and existing infrastructure.


In [20]:
user_query = "We are just starting with MTSS. What are the core features of the framework. Where should we begin? "
role = 'School_Administrators'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

Implementing a Multi-Tiered System of Supports (MTSS) is a strategic approach to enhancing educational outcomes for all students. It integrates academic, behavioral, and social-emotional support in a comprehensive framework. Here are the core features of the MTSS framework and some guidance on where to begin:

### Core Features of the MTSS Framework:

1. **Tiered Delivery System:**
   - **Tier 1:** Universal supports for all students to promote academic success and well-being through high-quality instruction and proactive strategies.
   - **Tier 2:** Targeted interventions for students who need additional support beyond the universal programs.
   - **Tier 3:** Intensive, individualized interventions for students who require more specialized assistance.

2. **Data-Informed Decision Making:**
   - Regular collection and analysis of data on student performance and behavior to guide decisions at all levels of the MTSS framework.

3. **Evidence-Based Practices:**
   - Implementation of inst

In [21]:
user_query = "I need family voice that represents our diverse school community.  I have lots of input from a vocal but small minority of parents expressing concerns about integrating positive health and wellness (SEL) in our Tier 1 curriculum. I believe most families are in favor of this, but they are not as vocal. What can I do to get more diverse voices especially around positive health and wellness (SEL)?  "
role = 'School_Administrators'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

To gather a more diverse range of family voices, especially concerning the integration of positive health and wellness (Social Emotional Learning or SEL) in your Tier 1 curriculum, you can employ several strategies to ensure broad and inclusive family engagement. Here are some practical steps you can take:

1. **Conduct Surveys and Questionnaires:**
   - Develop and distribute surveys or questionnaires that ask specific questions about families' views on integrating SEL into the curriculum. Ensure these are available in multiple languages and accessible formats to accommodate all families. Online platforms can be used for easy distribution and collection of responses.

2. **Host Focus Groups:**
   - Organize focus group sessions with parents and guardians from diverse backgrounds. These sessions can provide deeper insights into their thoughts and feelings about SEL. Ensure that the groups are facilitated by someone who can create an inclusive atmosphere and encourage open dialogue.

3.

In [9]:
user_query = "I need family voice that represents our diverse school community.  I have lots of input from a vocal but small minority of parents expressing concerns about integrating positive health and wellness (SEL) in our Tier 1 curriculum. I believe most families are in favor of this, but they are not as vocal. What can I do to get more diverse voices especially around positive health and wellness (SEL)? "
role = 'School_Administrators'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


To gather a more diverse range of family voices on integrating positive health and wellness (Social Emotional Learning, SEL) into your Tier 1 curriculum, you can employ several strategies to ensure inclusivity and broad participation. Here are some actionable steps:

1. **Diverse Communication Channels**: Use multiple communication methods to reach all families. This includes emails, school newsletters, text messages, social media, and physical flyers. Consider translating communications into the primary languages spoken at home by students' families.

2. **Flexible Meeting Times**: Schedule meetings at various times, including evenings and weekends, to accommodate different work schedules. Offer virtual options for those who cannot attend in person.

3. **Targeted Outreach**: Engage directly with community groups, cultural associations, and religious organizations that are integral to your diverse community. They can help relay information and encourage participation from their member

In [10]:
user_query = "We have seen an uptick in students coming to the school counselors expressing issue related to anxiety and depression. We currently refer to a community-based clinic for support but the students are not making it to their appointments for various reasons.  We would like to hire a clinician to work in the building. However, there is a group of parents in our district who are resistant to our school providing mental health services.  Can you tell me why it is important to deliver mental health services in schools? How can we increase buy in from the parents in our district?"
role = 'School_Administrators'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

Delivering mental health services in schools is crucial for several reasons, and gaining parental buy-in is essential for the success of such initiatives. Here’s a detailed explanation of why it's important to provide these services in schools and strategies to increase parental support:

### Importance of Delivering Mental Health Services in Schools

1. **Accessibility and Convenience**: Schools provide a central, accessible location where students spend a significant portion of their day. Offering mental health services on-site eliminates barriers such as transportation, scheduling conflicts, and the stigma of visiting a mental health clinic.

2. **Early Identification and Intervention**: School-based mental health professionals can identify and address emotional and behavioral issues early, before they escalate into more serious problems. This early intervention can lead to better outcomes for students.

3. **Academic Performance**: Mental health significantly impacts learning and c

In [11]:
user_query = "Why do schools struggle to integrate school behavioral health services into their MTSS"
role = 'School_Administrators'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

Schools often face several challenges when trying to integrate school behavioral health services into their Multi-Tiered System of Supports (MTSS). These challenges can stem from various systemic, logistical, and cultural factors:

1. **Resource Limitations**: One of the primary challenges is the lack of adequate resources, including funding, personnel, and time. Schools may struggle to hire qualified mental health professionals due to budget constraints or a shortage of available professionals in the area.

2. **Professional Development**: There may be a lack of adequate training for staff on how to effectively integrate behavioral health services into the existing MTSS framework. This includes training on identifying mental health issues, using appropriate interventions, and understanding the legal and ethical considerations involved.

3. **Data Systems Integration**: Effective MTSS implementation requires robust data collection and analysis systems. Schools may find it challenging t

In [12]:
user_query = "Can you share with me, in a simple way, what RTI (Response to Intervention) is? How is it different than MTSS?"
role = 'School_Administrators'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

**Response to Intervention (RTI)** is a specific type of educational strategy aimed at helping students who are struggling academically. It uses a tiered approach to provide interventions at increasing levels of intensity. Here’s a simple breakdown of the tiers in RTI:

1. **Tier 1:** High-quality classroom instruction and screening for all students.
2. **Tier 2:** Targeted interventions for students who need more support than what is provided in Tier 1.
3. **Tier 3:** Intensive interventions and evaluations for students who continue to struggle after Tier 2 support.

**RTI** is primarily focused on academics, particularly in identifying and supporting students with learning disabilities.

**Multi-Tiered System of Supports (MTSS)**, on the other hand, is a broader framework that includes RTI but also encompasses other types of student support. MTSS covers academic, behavioral, social, and emotional interventions. It’s a whole-school, data-driven approach aimed at improving outcomes for

In [ ]:
user_query = "What are the key concepts in implementing an effective integrated MTSS?"
role = 'School_Administrators'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

In [ ]:
user_query = "What are the key concepts in implementing an effective integrated MTSS?"
role = 'School_Administrators'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

In [ ]:
user_query = "What are the key concepts in implementing an effective integrated MTSS?"
role = 'School_Administrators'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

## Clinical_Staff

In [13]:
user_query = "What is MTSS, PBIS, ISF, RTI?  All these acronyms are floating around.  I need succinct answers. My next meeting is in 15 minutes.  "
role = 'Clinical_Staff'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

Certainly! Here's a quick rundown of each acronym to help you in your meeting:

1. **MTSS (Multi-Tiered System of Supports)**: A comprehensive framework that provides a structure for delivering levels of support (tiers) to students based on their needs. It integrates academic, behavioral, and social-emotional supports.

2. **PBIS (Positive Behavior Interventions and Supports)**: A subset of MTSS focused specifically on behavior. It uses evidence-based strategies and interventions to improve school climate and reduce behavioral problems.

3. **ISF (Interconnected Systems Framework)**: This framework integrates mental health services with PBIS within the MTSS model. It aims to enhance collaboration between education and mental health systems to support the whole child.

4. **RTI (Response to Intervention)**: A part of the MTSS framework that specifically addresses academic interventions. It involves multiple tiers of interventions that become more targeted and intensive based on the stud

In [15]:
user_query = "I am new to my district and have been invited to join an MTSS team as clinical staff. I am not clear on what they mean when they say I am an MTSS Team member. What is expected of me on this team? "
role = 'Clinical_Staff'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

Welcome to your new role! Joining an MTSS (Multi-Tiered System of Supports) team as clinical staff is a significant opportunity to contribute to the holistic support of students in your district. As a member of the MTSS team, your role will be integral in addressing the mental health and wellness needs of students across all tiers of support. Here’s a breakdown of what is generally expected from you in this role:

### 1. **Attendance and Participation in MTSS Meetings**
   - You will be expected to attend all MTSS meetings. These meetings are crucial for discussing student progress, planning interventions, and coordinating efforts across different tiers of support.

### 2. **Data Preparation and Reporting**
   - You will prepare and report on both qualitative and quantitative data concerning students and families receiving intensive support. This involves tracking the progress and outcomes of interventions to inform future practices.

### 3. **Facilitation of Problem Solving**
   - Off

In [16]:
user_query = "List the responsibilities of an LPC, LMSW, LMFT within a school building integrated MTSS framework."
role = 'Clinical_Staff'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

In a school building integrated within a Multi-Tiered System of Supports (MTSS) framework, professionals such as Licensed Professional Counselors (LPC), Licensed Master Social Workers (LMSW), and Licensed Marriage and Family Therapists (LMFT) play crucial roles. These roles are centered around providing mental health support, facilitating communication between the school and families, and contributing to the overall well-being of students. Here are the responsibilities typically associated with each of these roles within an MTSS framework:

### Licensed Professional Counselors (LPC)
1. **Assessment and Screening**: Conduct universal screenings and assessments to identify students who may need additional mental health support.
2. **Direct Counseling Services**: Provide direct services to students, including individual and group counseling, to address emotional, behavioral, and social needs.
3. **Crisis Intervention**: Respond to school crises by providing immediate counseling support an

In [17]:
user_query = "I work in a large school district.  It seems like every student who has an issue, behavior, emotion, and/or mental health is immediately referred to me.  I can’t keep up.  What can I do to inform a more preventative approach in my school? "
role = 'Clinical_Staff'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

Addressing the overwhelming number of referrals for student issues related to behavior, emotions, and mental health in a large school district requires a strategic, systemic approach that emphasizes prevention and early intervention. Here are several steps you can take to foster a more preventative culture in your school:

1. **Advocate for a Multi-Tiered System of Supports (MTSS):**
   - **Educate and Collaborate:** Work with school administrators and staff to implement or strengthen an MTSS framework that includes clear criteria for referrals at each tier. This helps ensure that students receive appropriate, timely interventions based on the severity and nature of their needs.
   - **Tier 1 Interventions:** Focus on universal strategies that benefit all students, such as social-emotional learning (SEL) curricula, positive behavioral interventions and supports (PBIS), and school-wide mental health literacy programs.

2. **Professional Development:**
   - **Training for Staff:** Organi

In [18]:
user_query = "My school does not use a universal screening system. What screeners can I recommend for use that capture internalizing and externalizing behavior? Can you give me a list of all available screening tools and a description of each."
role = 'Clinical_Staff'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

Implementing a universal screening system for internalizing and externalizing behaviors in a school setting is a proactive step towards identifying students who may need additional support. Here’s a list of some widely recognized and evidence-based screening tools that can help in capturing these behaviors:

1. **Systematic Screening for Behavior Disorders (SSBD)**:
   - **Purpose**: Identifies students at risk for internalizing and externalizing behavior problems.
   - **Description**: The SSBD is a multi-gate screening process that involves teacher nominations, teacher-completed rating scales, and direct observations to identify students who may require further assessment and intervention.

2. **Behavior Assessment System for Children (BASC-3)**:
   - **Purpose**: Assesses a broad range of behaviors including internalizing and externalizing problems.
   - **Description**: BASC-3 includes teacher, parent, and self-report scales that measure various aspects of behavior and emotions. It

In [ ]:
user_query = "What is MTSS, PBIS, ISF, RTI?  All these acronyms are floating around.  I need succinct answers. My next meeting is in 15 minutes.  "
role = 'Clinical_Staff'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

In [ ]:
user_query = "What is MTSS, PBIS, ISF, RTI?  All these acronyms are floating around.  I need succinct answers. My next meeting is in 15 minutes.  "
role = 'Clinical_Staff'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

In [ ]:
user_query = "What is MTSS, PBIS, ISF, RTI?  All these acronyms are floating around.  I need succinct answers. My next meeting is in 15 minutes.  "
role = 'Clinical_Staff'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

In [ ]:
user_query = "What is MTSS, PBIS, ISF, RTI?  All these acronyms are floating around.  I need succinct answers. My next meeting is in 15 minutes.  "
role = 'Clinical_Staff'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

In [ ]:
user_query = "What is MTSS, PBIS, ISF, RTI?  All these acronyms are floating around.  I need succinct answers. My next meeting is in 15 minutes.  "
role = 'Clinical_Staff'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

In [ ]:
user_query = "What is MTSS, PBIS, ISF, RTI?  All these acronyms are floating around.  I need succinct answers. My next meeting is in 15 minutes.  "
role = 'Clinical_Staff'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

In [ ]:
user_query = "What is MTSS, PBIS, ISF, RTI?  All these acronyms are floating around.  I need succinct answers. My next meeting is in 15 minutes.  "
role = 'Clinical_Staff'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

In [ ]:
user_query = "What is MTSS, PBIS, ISF, RTI?  All these acronyms are floating around.  I need succinct answers. My next meeting is in 15 minutes.  "
role = 'Clinical_Staff'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)